In [1]:
import requests
from datetime import datetime
import time
import pandas as pd

In [2]:
# Please insert your own plaid keys

import sys
sys.path.append('../Gamefication-Retirement/KeysPlaid')

import Plaid_Keys

PLAID_CLIENT_ID = Plaid_Keys.PLAID_CLIENT_ID
PLAID_SECRET = Plaid_Keys.PLAID_SECRET
PLAID_ENV = Plaid_Keys.PLAID_ENV

### Defining our Functions

In [3]:
def get_access_token(public_token):
    payload = {
        'client_id': PLAID_CLIENT_ID,
        'secret': PLAID_SECRET,
        'public_token': public_token
    }
    response = requests.post(f'{PLAID_ENV}/item/public_token/exchange', json=payload)
    return response.json()['access_token']

def create_public_token():
    payload = {
        'client_id': PLAID_CLIENT_ID,
        'secret': PLAID_SECRET,
        'institution_id': 'ins_1',  # Sandbox institution ID
        'initial_products': ['transactions']
    }
    response = requests.post(f'{PLAID_ENV}/sandbox/public_token/create', json=payload)
    return response.json()['public_token']


def get_transactions(access_token, start_date, end_date):
    payload = {
        'client_id': PLAID_CLIENT_ID,
        'secret': PLAID_SECRET,
        'access_token': access_token,
        'start_date': start_date,
        'end_date': end_date,
        'options': {
            'count': 500,
        }
    }
    response = requests.post(f'{PLAID_ENV}/transactions/get', json=payload)
    return response.json()

def poll_transactions(access_token, start_date, end_date, max_retries=5, delay=5):
    for attempt in range(max_retries):
        response = get_transactions(access_token, start_date, end_date)
        if response.get('error_code') != 'PRODUCT_NOT_READY':
            return response
        time.sleep(delay)
    return response


### Now Lets get our transaction response and append to CSV

In [4]:
public_token = create_public_token()
access_token = get_access_token(public_token)

start_date = datetime(2018, 1, 1).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

transactions_response = poll_transactions(access_token, start_date, end_date)

In [5]:
transactions_df = transactions_response.get('transactions', [])

df = pd.DataFrame(transactions_df)

csv_filename = 'transactions.csv'
df.to_csv(csv_filename, index=False)